In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index


array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    # ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD" and attack_name!= "DF":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
x_test_adv_0_1.shape

(683826, 56)

In [6]:
from xgboost import XGBClassifier

print('xgb')
xgb100 = XGBClassifier()
xgb100.fit(x_test_adv_0_1, y_label_0_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf100 = RandomForestClassifier()
rf100.fit(x_test_adv_0_1, y_label_0_1)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt100 = DecisionTreeClassifier()
dt100.fit(x_test_adv_0_1, y_label_0_1)

xgb
RF
DT


DecisionTreeClassifier()

In [7]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [8]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb100.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Exclude3Attack/UNSW_Input100/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Exclude3Attack/UNSW_Input100/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [   202    327   2105   2273     45    451      1    161 108026    282
     98]
(202, 56) (202,)
Save 0 to baseline_Def1.npy
(327, 56) (327,)
Save 1 to baseline_Def2.npy
(2105, 56) (2105,)
Save 2 to baseline_Def3.npy
(2273, 56) (2273,)
Save 3 to baseline_Def4.npy
(45, 56) (45,)
Save 4 to baseline_Def5.npy
(451, 56) (451,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(161, 56) (161,)
Save 7 to baseline_Def8.npy
(108026, 56) (108026,)
Save 8 to baseline_Def9.npy
(282, 56) (282,)
Save 9 to baseline_Def10.npy
(98, 56) (98,)
Save 10 to baseline_Def11.npy
Execution Time: 0.477383 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [215927    292   2229   3540   1127   4605     65 113986     93     49]
(215927, 56) (215927,)
Save 0 to BIM_Def1.npy
(292, 56) (292,)
Save 1 to BIM_Def2.npy
(2229, 56) (2229,)
Save 2 to BIM_Def3.npy
(3540, 56) (3540,)
Save 3 to BIM_Def4.npy
(1127, 56) (1127,)
Save 4 to BIM_Def5.npy
(4605, 56) (4605

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf100.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude3Attack/UNSW_Input100/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude3Attack/UNSW_Input100/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [   471    509   2493   2025     90   1194      1    327 106433    285
    143]
(471, 56) (471,)
Save 0 to baseline_Def1.npy
(509, 56) (509,)
Save 1 to baseline_Def2.npy
(2493, 56) (2493,)
Save 2 to baseline_Def3.npy
(2025, 56) (2025,)
Save 3 to baseline_Def4.npy
(90, 56) (90,)
Save 4 to baseline_Def5.npy
(1194, 56) (1194,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(327, 56) (327,)
Save 7 to baseline_Def8.npy
(106433, 56) (106433,)
Save 8 to baseline_Def9.npy
(285, 56) (285,)
Save 9 to baseline_Def10.npy
(143, 56) (143,)
Save 10 to baseline_Def11.npy
Execution Time: 1.502113 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [215497    193   2530   3323    225   5200     83 114794     54     14]
(215497, 56) (215497,)
Save 0 to BIM_Def1.npy
(193, 56) (193,)
Save 1 to BIM_Def2.npy
(2530, 56) (2530,)
Save 2 to BIM_Def3.npy
(3323, 56) (3323,)
Save 3 to BIM_Def4.npy
(225, 56) (225,)
Save 4 to BIM_Def5.npy
(5200, 56) (52

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt100.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Exclude3Attack/UNSW_Input100/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Exclude3Attack/UNSW_Input100/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [   502    545   2688   2131     99   1306      2    335 105892    300
    171]
(502, 56) (502,)
Save 0 to baseline_Def1.npy
(545, 56) (545,)
Save 1 to baseline_Def2.npy
(2688, 56) (2688,)
Save 2 to baseline_Def3.npy
(2131, 56) (2131,)
Save 3 to baseline_Def4.npy
(99, 56) (99,)
Save 4 to baseline_Def5.npy
(1306, 56) (1306,)
Save 5 to baseline_Def6.npy
(2, 56) (2,)
Save 6 to baseline_Def7.npy
(335, 56) (335,)
Save 7 to baseline_Def8.npy
(105892, 56) (105892,)
Save 8 to baseline_Def9.npy
(300, 56) (300,)
Save 9 to baseline_Def10.npy
(171, 56) (171,)
Save 10 to baseline_Def11.npy
Execution Time: 0.144141 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [188156   2033   5464  54136   2961   6716    825  74559    249   6814]
(188156, 56) (188156,)
Save 0 to BIM_Def1.npy
(2033, 56) (2033,)
Save 1 to BIM_Def2.npy
(5464, 56) (5464,)
Save 2 to BIM_Def3.npy
(54136, 56) (54136,)
Save 3 to BIM_Def4.npy
(2961, 56) (2961,)
Save 4 to BIM_Def5.npy
(6716, 5

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}100")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_Exclude3Attack/UNSW_Input100/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input100_Exclude3Attack.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB100,202,327,2105,2273,45,451,1,161,108026,282,98,113971
1,XGB100,215927,292,2229,3540,1127,4605,0,65,113986,93,49,341913
2,XGB100,218258,180,1876,5786,923,2312,0,406,112032,62,78,341913
3,XGB100,215927,292,2229,3540,1127,4605,0,65,113986,93,49,341913
4,XGB100,304195,582,1494,5505,1792,1318,0,88,26716,178,45,341913
5,XGB100,201150,971,4847,7167,939,2143,0,475,123252,684,285,341913
6,XGB100,620,970,6389,5909,135,1227,3,483,325091,800,286,341913
7,XGB100,82360,61,1337,1535,51,121,2,11,256236,130,69,341913
8,XGB100,213967,727,1093,6008,1102,2072,0,31,116852,24,37,341913
9,XGB100,214920,227,2344,5601,1750,4515,0,46,112372,72,66,341913
